# Airada Taweeyossak 6030830121

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import folium as fo
import matplotlib.cm as cm
import matplotlib.colors as colors
from folium import plugins
from folium.plugins import MarkerCluster

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/Maplub/accident_sample_bkk/main/event2019_BKK.csv')
data

,title,latitude,longitude
0,อุบัติเหตุ ถนนรัชดาภิเษก ขาเข้า,13.756988,100.565255
1,อุบัติเหตุ ถนนทางรถไฟเก่าสายปากน้ำ,13.717649,100.565232
2,อุบัติเหตุ ถนนเจริญกรุง ขาเข้า,13.728361,100.515878
3,อุบัติเหตุ กรุงเทพมหานคร,13.753568,100.580294
4,อุบัติเหตุ แยกคลองเตย,13.720805,100.557668
...,...,...,...
3061,อุบัติเหตุ ถนนเพชรบุรี,13.749695,100.548232
3062,อุบัติเหตุ ห้วยขวาง กรุงเทพมหานคร,13.753643,100.580295
3063,อุบัติเหตุ ห้วยขวาง กรุงเทพมหานคร,13.753630,100.580293
3064,อุบัติเหตุ เจริญรัถ 8 (ซอยอยู่ดี),13.704420,100.516291


In [3]:
longitude =  100.5538272857666
latitude = 13.734465925243732
station_map = fo.Map(
                location = [latitude, longitude], 
                zoom_start = 14)
latitudes = list(data.latitude)
longitudes = list(data.longitude)
labels = list(data.title)

for lat, lng, label in zip(latitudes, longitudes, labels):
    fo.Circle(
      location = [lat, lng],
      radius = 1,
      color = 'blue' 
     ).add_to(station_map)
station_map

In [4]:
points = np.array(data[['longitude','latitude']])

## Hierarchical Clustering

In [5]:
from sklearn.cluster import AgglomerativeClustering as Agg
model = Agg(n_clusters=600).fit(points)
agg_label=model.labels_

In [6]:
from scipy.cluster.hierarchy import dendrogram
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [7]:
from sklearn.metrics import silhouette_samples, silhouette_score
silhouette_score(points, agg_label)

0.6209576533769091

เมื่อทำการลอง clustering แบบ Hierarchical ดูพบว่า ถ้าหากต้องการแบ่งจำนวน cluster ตามค่า silhouette แล้วจะแบ่งได้ประมาณ 600 ค่าถึงจะได้ค่า silhouette ที่ดีสุด ผลนี้เป็นไปได้ว่าข้อมูลมีการซ้ำกัน และมีข้อมูลที่ม่ซ้ำกันอยู่ประมาณ 600 แบบ

## Plot Heat Map

In [8]:
from folium.plugins import HeatMap
data['count'] = 1
base_map = station_map
HeatMap(data = data[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=9, max_zoom=13).add_to(base_map)

In [9]:
station_map